In [1]:
import pandas as pd
import numpy as np

import gspread

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [ ]:
gc = gspread.service_account()

In [67]:
cb = gc.open_by_key('1tjosPLovf5tLe0kiDsO6hEu2aSkkOpeapAlbf0PnDTY')
bills = cb.worksheet('Sheet1')
bills = bills.get_all_records()
bills = pd.DataFrame(bills)

In [ ]:
bills_uncat = pd.read_csv('../../cc_legislation/data/bills_with_cluster_count.csv')

In [69]:
vectorizer = TfidfVectorizer()

In [52]:
bills.columns

Index(['MatterSponsorMatterId', 'sponsors', 'cluster_0', 'cluster_1',
       'cluster_2', 'cluster_3', 'cluster_4', 'primary_sponsor_cluster',
       'EventItemMatterStatus', 'EventItemMatterFile', 'EventItemMatterName',
       'category'],
      dtype='object')

In [70]:
# only bills with a category
bills_test = bills[bills['category'].notnull()]

In [71]:
bills_test = bills_test[bills_test['category'].str.strip() != '']

In [72]:
bills_test.shape

(323, 12)

In [64]:
bills_test['category'].value_counts()

category
Streets                  35
Public Health            32
Tenant Protection        21
Worker Protection        21
Public Welfare/Safety    19
Schools                  18
Government Service       17
Good Governance          17
Gender and Sexuality     15
Business                 15
Climate                  15
Incarceration/Jails      15
Policing                 14
Transit                  13
Homelessness             12
Sanitation               12
Housing                  11
Animal Welfare            9
Consumer Protection       7
Accessability             5
Name: count, dtype: int64

In [73]:
X = vectorizer.fit_transform(bills_test['EventItemMatterName'])
y = bills_test['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [74]:
model = MultinomialNB()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.13846153846153847


In [75]:
# predict the category of the uncategorized bills
X_uncat = vectorizer.transform(bills['EventItemMatterName'])
y_uncat = model.predict(X_uncat)
bills['guess_catagory'] = y_uncat

In [76]:
bills['guess_catagory'].head()

0    Tenant Protection
1              Streets
2              Streets
3              Streets
4              Streets
Name: guess_catagory, dtype: object

In [80]:
bills.groupby('primary_sponsor_cluster')['guess_catagory'].value_counts()

primary_sponsor_cluster  guess_catagory       
0                        Streets                  216
                         Public Health             81
                         Schools                    7
                         Tenant Protection          4
                         Gender and Sexuality       4
                         Worker Protection          2
                         Public Welfare/Safety      2
                         Government Service         2
                         Business                   1
                         Sanitation                 1
1                        Streets                  136
                         Public Health             26
                         Government Service         4
                         Tenant Protection          2
                         Sanitation                 2
                         Public Welfare/Safety      2
                         Worker Protection          1
                         Homelessne